In [2]:
import numpy as np
import pandas as pd

There are a number of basic operations for rearranging tabular data. These are alter‐
natingly referred to as reshape or pivot operations.

## Reshaping with Hierarchical Indexing

Hierarchical indexing provides a consistent way to rearrange data in a DataFrame. There are two primary actions:

    stack: This “rotates” or pivots from the columns in the data to the rows
    unstack: This pivots from the rows into the columns

In [3]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [5]:
# Using the stack method on this data pivots the columns into the rows, producing a Series:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [6]:
# From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with unstack:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [7]:
# By default the innermost level is unstacked (same with stack). 
# You can unstack a different level by passing a level number or name:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [8]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


__Unstacking might introduce missing data if all of the values in the level aren’t found in each of the subgroups:__

In [10]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [11]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [12]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [13]:
# Stacking filters out missing data by default, so the operation is more easily invertible:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [14]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

__When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result:__

In [15]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [16]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [17]:
# When calling stack, we can indicate the name of the axis to stack:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## Pivoting “Long” to “Wide” Format

A common way to store multiple time series in databases and CSV is in so-called long
or stacked format. Let’s load some example data and do a small amount of time series
wrangling and other data cleaning:

In [18]:
data = pd.read_csv('macrodata.csv')

In [20]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [25]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [26]:
ldata[:10]
# This is the so-called long format for multiple time series, or other observational data
# with two or more keys (here, our keys are date and item). Each row in the table repre‐
# sents a single observation.

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


Data is frequently stored this way in relational databases like MySQL, as a fixed
schema (column names and data types) allows the number of distinct values in the
item column to change as data is added to the table. In the previous example, date
and item would usually be the primary keys (in relational database parlance), offering
both relational integrity and easier joins. In some cases, the data may be more diffi‐
cult to work with in this format; you might prefer to have a DataFrame containing
one column per distinct item value indexed by timestamps in the date column. Data‐
Frame’s pivot method performs exactly this transformation:

In [31]:
pivoted = ldata.pivot(index='date',columns='item',values='value')
pivoted
# The first two values passed are the columns to be used respectively as the row and
# column index, then finally an optional value column to fill the DataFrame.

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


__Suppose you had two value columns that you wanted to reshape simultaneously:__

In [32]:
ldata['value2'] = np.random.randn(len(ldata))

In [34]:
# By omitting the last argument, you obtain a DataFrame with hierarchical columns:
pivoted = ldata.pivot(index='date',columns='item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.426496 -0.792983   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.485925 -0.144464   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.737739  0.835118   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.057385  1.885555   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.610262  0.413586   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.717191  
1959-06-30 23:59:59.999999999 -0.258699  
1959-09-30 23:59:59.999999999  0.671639  
1959-12-31 23:59:59.999999999  0.256333  
1960-03-31 23:59:59.999999999 -0.149008

In [35]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [36]:
# Note that pivot is equivalent to creating a hierarchical index using set_index followed by a call to unstack:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.426496 -0.792983   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.485925 -0.144464   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.737739  0.835118   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.057385  1.885555   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.610262  0.413586   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2  0.108224 -1.015138   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.057778 -0.344483   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.717191  
1959-06-30 23:59:59.999999999 -0.258699  
1959-09-30 23:59:59.999999999  0.671639  
1959-12-31 23:59:59.999999999  0.256333  
1960-03-31 23:59:59.999999999 -0.149008  
1960-06-30 23:59:59.999999999  1.313179  
1960-09-30 23:59:59.999999999  0.205642

In [ ]:
ldata.pivot?

## Pivoting “Wide” to “Long” Format

An inverse operation to pivot for DataFrames is pandas.melt. Rather than trans‐
forming one column into many in a new DataFrame, it merges multiple columns into
one, producing a DataFrame that is longer than the input. Let’s look at an example:

In [37]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


The 'key' column may be a group indicator, and the other columns are data values.
When using pandas.melt, we must indicate which columns (if any) are group indica‐
tors. Let’s use 'key' as the only group indicator here:

In [38]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [39]:
# Using pivot, we can reshape back to the original layout:
reshaped = melted.pivot(index='key',columns='variable',values='value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [40]:
# Since the result of pivot creates an index from the column used as the row labels, we
# may want to use reset_index to move the data back into a column:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [41]:
# You can also specify a subset of columns to use as value columns:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [42]:
# pandas.melt can be used without any group identifiers, too:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [43]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
